<a href="https://colab.research.google.com/github/claykaufmann/cs302-final-project/blob/main/FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# CS 302 Final Project

Yousef Khan, Jaden Varin, Clay Kaufmann

For our final project, we decided to model the path of a photon in the sun, based on different models of what the inside of the sun may be. To do this, we built an event-driven model, where instead of iterating over time, we iterate over the interactions a photon has with hydrogen atoms. So, every step of a loop is the next interaction. The basic idea is that at each step, you calculate the estimated distance until the photon has another interaction, and randomly select a direction to take. Then in the next iteration of the loop, you do this again.


In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly_resampler import FigureResampler
import plotly.io as pio
pio.renderers
pio.renderers.default = "notebook_connected"
from mpl_toolkits import mplot3d
import os
from IPython.display import clear_output
import csv


np.random.seed()

## Photon Class

The photon class represents a single photon.


In [ ]:
class Photon:
    """
    a single photon in space
    """

    def __init__(self) -> None:
        """
        The constructor simply initializes the history list, to keep track of where the photon has been.
        """
        self.history = [[0, 0, 0]]

    def next_distance(self, function):
        """
        calculate the next distance for the photon to travel before an interaction

        Parameters
        ----------
        self: the photon class
        function: the modeling function that calculates the distance
        """
        res = function()

        return res

    def next_loc(self, distance):
        """
        get the next photon location

        Parameters
        ----------
        self: self
        distance: float, representing the radius of the sphere for possible next location
        """
        z = np.random.uniform(distance * -1, distance)
        
        # phi is the angle, which is the "longitude"
        phi = np.random.uniform(0, 2 * np.pi)

        # calculate the theta
        theta = np.random.uniform(0, 2 * np.pi)

        # gen points
        x = distance * np.sin(phi) * np.cos(theta);
        y = distance * np.sin(phi) * np.sin(theta);
        
        # update tracking location
        # take previous iteration, append values
        final_x = self.history[-1][0] + x
        final_y = self.history[-1][1] + y
        final_z = self.history[-1][2] + z

        # append this new location to the photon's history
        self.history.append([final_x, final_y, final_z])


## Equations and Constants

In this cell, we establish core constants and equations we will use for our modeling functions.


In [ ]:
# Numbers assume hydrogen atoms
# sources:
# https://observatory.astro.utah.edu/sun.html
# http://solar-center.stanford.edu/vitalstats.html
# https://physicsanduniverse.com/random-walk-photon/
# https://www.compadre.org/osp/items/detail.cfm?ID=11349


def kgm3_to_cmg3(val):
    """
    convert data from kg/m^3 to cm/g^3
    """
    return val / 1000

def mfp(density, opacity):
    """
    calculate mean free path with given hydrogen density and opacity

    Parameters
    ----------
    density: a given hydrogen density in the sun
    opacity: the opacity of the sun
    """

    return 1.0 / (density * opacity)

def calc_escape_time(R, l):
    """
    calculate the time it takes for a photion to reach sun's surface in seconds
    """
    # time it takes for photon to reach sun's surface in seconds
    return np.square(R) / (mfp * C)

def seconds_to_years(secs):
    """
    convert seconds to years
    """
    return int(secs/31_556_952)

def distance(coordinates):
    """
    calculate the distance in 3D space from the origin

    Parameters
    ----------
    coordinates: list, np.ndarray, tuple
        a list of coordinates
    """
    x, y, z = coordinates
    return np.sqrt(x**2 + y**2 + z**2)


def run_eq(x):
    """
    helper function for get_density, calculates density at distance from center in percentage of total distance
    """
    g_cm3 = 519.0*(x**4) - 1630.0*(x**3) + 1844.0*(x**2) - 889.0*x + 155.0
    return g_cm3 * SCALE * 1000 # convert from g/cm3 to kg/m3

def get_density(bins=0):
    """
    calculate the density of sun

    SOURCE: https://spacemath.gsfc.nasa.gov/Calculus/6Page102.pdf
    """
    if bins == 0:
        bins = R
    return [run_eq(i / bins) for i in range(int(bins))]


def print_dist(n, dist):
    clear_output(wait=True)
    print(f'N: {n}')
    bins = R / 50
    s = "=" * int(dist / bins)
    space = " " * int(50 - (dist/bins))
    print("Start:", "|" + s + ">", space + "|", "Finish")
    

### CONSTANTS ###
# DENSITY VALUES TO BE USED IN TESTING
C = 3.0e8 # speed of light m/s
opacity = 3.0 # opacity m^2/kg
R = 7.0e8 # sun's radius in meters
# SCALE = 1/1_000_000_000 # scale the sun so that the simulation actually runs
SCALE = 1/15_000_000_000 # scale the density

PLOTTING = False

average_density_results = {}
decreasing_density_results = {}
discrete_density_results = {}


## Plotting Functions

The following functions create the visualization.


In [ ]:
def create_sun(size, clr, dist=0, opacity=1):
    """
    create a yellow sphere, with a given size and opacity
    """
    # Set up 100 points. First, do angles
    theta = np.linspace(0,2*np.pi,100)
    phi = np.linspace(0,np.pi,100)
    
    # Set up coordinates for points on the sphere
    x0 = dist + size * np.outer(np.cos(theta),np.sin(phi))
    y0 = size * np.outer(np.sin(theta),np.sin(phi))
    z0 = size * np.outer(np.ones(100),np.cos(phi))
    
    # Set up trace
    trace = go.Surface(x=x0, y=y0, z=z0, colorscale=[[0,clr], [1,clr]], opacity=opacity)
    trace.update(showscale=False)

    return trace

def create_photon(x, y, z, clr='white', wdth=2):
    """
    create a photon track using a 3d scatter plot

    Parameters
    ----------
    x: np.ndarray
        an array of points to plot in the x dim
    y: np.ndarray
        an array of points to plot in the y dim
    z: np.ndarray
        an array of points to plot in the z dim
    """
    # build trace
    trace = go.Scatter3d(x=x, y=y, z=z, line=dict(color=clr, width=wdth), marker=dict(size=0.1))

    return trace

def plot_photon_and_sun(photon: Photon, cut_down_size=None, mode=None, plot_sun=True, sun_size=R * SCALE):
    """
    This function plots the photon and sun with Plotly Express

    Parameters
    ----------
    photon: Photon
        a photon class object
    cut_down_size: tuple(int, int)
        the amount of datapoints to plot to, i.e 1000 plots to the 1000th datapoint
    """
    photon_track = np.array(photon.history)

    if cut_down_size:
        photon_track = photon_track[cut_down_size[0]:cut_down_size[1]]

    layout = go.Layout(
        autosize=False,
        width=700,
        height=700,
        margin=go.layout.Margin(
            l=50,
            r=50,
            b=100,
            t=100,
            pad = 4
        )
    )

    # create figure resampler, for more efficient plotting
    fig = FigureResampler(go.Figure(layout=layout))

    # create the sun
    if plot_sun:
        sun = create_sun(sun_size, '#ffff00', 0, 0.2) # Sun
        fig.add_trace(sun)

    # create the photon
    photon_trace = create_photon(photon_track[:,0], photon_track[:,1], photon_track[:,2], clr='red')

    # add traces to the figure
    fig.add_trace(photon_trace)

    # show the plot
    fig.show_dash(mode=mode)

def pyplot_photon_path(photon: Photon):
    """
    Plot the photon path in matplotlib
    Not an interactible chart, but is static and is easier to show large datasets.

    Parameters
    ----------
    photon: Photon
        a photon object
    """

    photon_track = np.array(photon.history)
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax = plt.axes(projection='3d')
    ax.scatter3D(photon_track[:,0], photon_track[:,1], photon_track[:,2])

    plt.show()

## The Models

We now implement and build our different models.

### Model 1: Average Density

The average density model assumes that the inside of the sun has one single density. We base all the photon interactions off of this single value.

#### Pseudocode

```
opacity = 3.0
density = 1408.0
l = 1.0 / (opacity * density)
photon_path = []
p = photon initialized at Sun’s center
while p position < Sun’s radius:
	update p position to the point of next interaction
	calculate new p direction
	append location to photon_path
endWhile
```


In [ ]:
1/0
sun_avg_density = 1408.0*SCALE # average density of the sun - kg/m^3
l = mfp(sun_avg_density, opacity) # mean free path using average density and opacity
p = Photon()
N = 0 # time steps initialization
while distance(p.history[-1]) < R:
    if N % 100_000 == 0:
        print_dist(N, distance(p.history[-1]))
        print(f'Distance: {distance(p.history[-1])} MFP: {l}')

#     if N % 1_000_000 == 0 and N != 0:
#         break

    N += 1
    p.next_loc(l)
    average_density_results[N] = distance(p.history[-1])
    

In [ ]:
try:
    with open('average_density_model.csv', 'w') as csv_file:  
        writer = csv.writer(csv_file)
        for key, value in average_density_results.items():
            writer.writerow([key, value])
except IOError:
    print("I/O error")

#### Plot the results


In [ ]:
1/0
if PLOTTING:
    plot_photon_and_sun(p, (0, 100_000), plot_sun=True)

### Model 2: Linear Decreasing Density

The linearly decreasing density model assumes a linear decrease in density from the core out to the surface of the sun.

#### Pseudocode

```
opacity = 3.0
density = [linearly decreasing values]
photon_path = []
p = photon initialized at Sun’s center
while p position< Sun’s Radius
	update density value at current radius
	update l to use current density
	update p position based on l
	calculate new p direction
	append location to photon_path
endWhile

```


In [ ]:
bins = 100_000
accurate_decreasing_density = get_density(bins)

In [ ]:
p_1 = Photon()
N = 0 # time steps initialization
while distance(p_1.history[-1]) < R:
        
    if N % 100_000 == 0 and N != 0:
        print_dist(N, distance(p_1.history[-1]))
        print(f'Distance: {distance(p_1.history[-1])} MFP: {l}')
    N += 1
    # updating density based on distance from center
    d = int(distance(p_1.history[-1]) / (R / bins))

    # update l to reflect current density
    l = mfp(accurate_decreasing_density[d], opacity)
    
    # update photon position based on new mean free path
    p_1.next_loc(l)

    decreasing_density_results[N] = distance(p_1.history[-1])


In [ ]:
try:
    with open('decreasing_density_model.csv', 'w') as csv_file:  
        writer = csv.writer(csv_file)
        for key, value in decreasing_density_results.items():
            writer.writerow([key, value])
except IOError:
    print("I/O error")

#### Plotting


In [ ]:
1/0
if PLOTTING:
    plot_photon_and_sun(p_1)

### Model 3: Discretized Density

The discretized density model follows the intution of setting discrete zones in the sun with specific densities. When a photon is in a specific zone, it has a different average distance until an interaction, as compared to other zones.

#### Pseudocode

```
opacity = 3.0
density = [core, radiative, convective]
photon_path = []
p = photon initialized at Sun’s center
While p position < Sun’s radius
	if  p in core:
update l to use core density
	endIf
	else if p in radiative layer:
update l to use radiative density
	endIf
	else if p in convective layer:
update l to use convective density
	endIf
	update p position based on l
	calculate new p direction
append location to photon_path
endWhile
```


In [ ]:
num_radiative_sections = 100
sun_layer_densities = {'core': 1.622e5,
                       'radiative': np.linspace(20_000, 200, num=num_radiative_sections),
                       'convective': 200} # values are approximate - kg/m^3
p_2 = Photon()
N = 0 # time steps initialization
l = mfp(sun_layer_densities['core'], opacity) # initialize mean free path to core density
while distance(p_2.history[-1]) < R:
    if N % 100_000 == 0:
        print_dist(N, distance(p_2.history[-1]))
        print(f'Distance: {distance(p_2.history[-1])} MFP: {l}')
    N += 1
    d = distance(p_2.history[-1]) + l # add l to see where the photon is going
    if d < R*0.25:
        # use densities for Sun's core
        density = sun_layer_densities['core']
    elif R*0.25 < d < R*0.7:
        # determine the size in meters of each chunk of the radiative zone
        chunk_size = (R*0.45 / num_radiative_sections)
        # determine how many chunks into the radiative zone the photon is
        dist = int((d - R*0.25) / chunk_size)
        # use densities for Sun's radiative zone at chunk index
        density = sun_layer_densities['radiative'][dist]
    else:
        # use densities for Sun's convective zone
        density = sun_layer_densities['convective']
    
    # update mean free path based on new density
    l = mfp(density*SCALE, opacity)

    # update photon position based on new mean free path
    p_2.next_loc(l)

    discrete_density_results[N] = distance(p_2.history[-1])

In [ ]:
try:
    with open('discrete_density_model.csv', 'w') as csv_file:  
        writer = csv.writer(csv_file)
        for key, value in discrete_density_results.items():
            writer.writerow([key, value])
except IOError:
    print("I/O error")

In [ ]:
if PLOTTING:
    plot_photon_and_sun(p_2, (0, 100000))

# **Plotting**


### **Line Graphs**


In [ ]:
with open('average_density_model.csv') as csv_file:
    reader = csv.reader(csv_file)
    average_density_model = dict(reader)

with open('decreasing_density_model.csv') as csv_file:
    reader = csv.reader(csv_file)
    decreasing_density_model = dict(reader)
    
# with open('discrete_density_model.csv') as csv_file:
#     reader = csv.reader(csv_file)
#     discrete_density_model = dict(reader)

In [ ]:
plt.plot(average_density_model.keys(), average_density_model.values(), label="Average Density Model")
plt.plot(decreasing_density_model.keys(), decreasing_density_model.values(), label="Linear Decreasing Density Model")
# plt.plot(discrete_density_model.keys(), discrete_density_model.values(), label="Discrete Density Model")
plt.legend()
# plt.yticks(np.arange(0, int(R), 1_000_000))
# plt.xticks(np.arange(0, max(len(average_density_model), 0), 100))
plt.xlabel("Steps")
plt.ylabel("Distance from center")
plt.title("Distance vs Time")
plt.show()